In [1]:
!pip install datasets
!pip install tokenizers
!pip install torchmetrics
!pip install transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
%cp -av /content/drive/MyDrive/Assignments/pizza_steak_sushi /content/DATA

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
'/content/drive/MyDrive/Assignments/pizza_steak_sushi' -> '/content/DATA'
'/content/drive/MyDrive/Assignments/pizza_steak_sushi/train' -> '/content/DATA/train'
'/content/drive/MyDrive/Assignments/pizza_steak_sushi/train/steak' -> '/content/DATA/train/steak'
'/content/drive/MyDrive/Assignments/pizza_steak_sushi/train/steak/225990.jpg' -> '/content/DATA/train/steak/225990.jpg'
'/content/drive/MyDrive/Assignments/pizza_steak_sushi/train/steak/922752.jpg' -> '/content/DATA/train/steak/922752.jpg'
'/content/drive/MyDrive/Assignments/pizza_steak_sushi/train/steak/2129685.jpg' -> '/content/DATA/train/steak/2129685.jpg'
'/content/drive/MyDrive/Assignments/pizza_steak_sushi/train/steak/2163079.jpg' -> '/content/DATA/train/steak/2163079.jpg'
'/content/drive/MyDrive/Assignments/pizza_steak_sushi/train/steak/3245622.jpg' -> '/content/DATA/train/steak/3245622.jpg'
'/con

In [1]:
from transformer import Transformer
from BERT_main import *
from GPT_main import *
from config import *
from BERT_Dataset import *


import torch.optim as optim
import torch.nn as nn
import numpy as np
import torch
import re

from collections import Counter
from os.path import exists

In [2]:
def main(model_type):

  if model_type == 'bert':

      config = get_bert_config()
      print('loading text...')
      sentences = load_sentences('/content/training.txt')
      print('tokenizing sentences...')
      special_chars = ',?;.:/*!+-()[]{}"\'&'
      sentences = tokenize_sentences(sentences, special_chars)
      print('creating/loading vocab...')
      vocab = create_or_load_vocab(sentences, '/content/vocab.txt', config['n_vocab'])
      print('creating dataset...')
      dataset = BERTDataset(sentences, vocab, config['seq_len'])
      data_loader = torch.utils.data.DataLoader(dataset, shuffle=True, drop_last=True, pin_memory=True, batch_size=config['batch_size'])
      print('initializing model...')
      inner_ff_size = config['embed_size'] * 4
      model = Transformer('bert', config['n_code'], config['n_heads'], config['embed_size'], inner_ff_size, len(dataset.vocab), config['seq_len'], config['dropout']).cuda()
      print(model)
      print('initializing optimizer and loss...')
      optim_kwargs = config["optim_kwargs"]
      optimizer = optim.Adam(model.parameters(), **optim_kwargs)
      loss_model = nn.CrossEntropyLoss(ignore_index=dataset.IGNORE_IDX)
      print('training...')
      print_each = 10
      num_iterations = 10000
      train_model_bert(model, data_loader, optimizer, loss_model, num_iterations, print_each)
      print('saving embeddings...')
      save_embeddings(model, dataset, 3000, 'values.tsv', 'names.tsv')
      print('end')

  elif model_type == 'gpt':

    config = get_gpt_config()
    path_to_data = "/content/english.txt"
    train_data, val_data, vocab_size,tokenizer = load_and_tokenize_data(path_to_data)
    model = Transformer('gpt',6,6,768,0,vocab_size,64,0.2)
    m = model.to(config['DEVICE'])
    optimizer = torch.optim.AdamW(m.parameters(), lr=config['LEARNING_RATE'])
    print("Model with {:.2f}M parameters".format(sum(p.numel() for p in model.parameters()) / 1e6))
    train_model_gpt(model, optimizer, train_data, val_data)
    context = torch.zeros((1, 1), dtype=torch.long, device=config['DEVICE'])
    generated_output = generate_output(model, tokenizer, context)
    print(generated_output)

  elif model_type == 'vit':

    image_path = "/content/DATA/pizza_steak_sushi"
    train_dir = image_path + "/train/"
    test_dir = image_path + "/test/"
    print(train_dir)
    train_data = datasets.ImageFolder(train_dir, transform=manual_transforms)
    test_data = datasets.ImageFolder(test_dir, transform=manual_transforms)
    class_names = train_data.classes
    device = "cuda" if torch.cuda.is_available() else "cpu"
    train_dataloader = DataLoader(
        train_data,
        batch_size=10,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )

    test_dataloader = DataLoader(
        test_data,
        batch_size=10,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
    )
    train_dataloader
    # Initialize the ViT model
    vit = ViT()

    # Setup the optimizer
    optimizer = torch.optim.Adam(params=vit.parameters(),
                                 lr=3e-3,
                                 betas=(0.9, 0.999),
                                 weight_decay=0.3)

    # Setup the loss function
    loss_fn = torch.nn.CrossEntropyLoss()

    # Train the model
    results = VIT_engine.train(model=vit,
                               train_dataloader=train_dataloader,
                               test_dataloader=test_dataloader,
                               optimizer=optimizer,
                               loss_fn=loss_fn,
                               epochs=5,
                               device=device)


In [3]:
if __name__ == "__main__":
    main('bert')

loading text...
tokenizing sentences...
creating/loading vocab...
creating dataset...
initializing model...
Transformer(
  (embeddings): Embedding(23948, 128)
  (pe): PositionalEmbedding()
  (encoders): ModuleList(
    (0-7): 8 x EncoderLayer(
      (mha): MultiHeadAttention(
        (linear): Linear(in_features=128, out_features=384, bias=True)
        (out): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (linear1): Linear(in_features=128, out_features=512, bias=True)
        (linear2): Linear(in_features=512, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
  )
  (norm): LayerNorm((128,), eps=1e-05, elementwise_affine